# An introduction to Python Coroutine

## WHAT

### A function and a generater

In [ ]:
def fun():
    return 'fun'

def gen():
    yield 'gen'

In [ ]:
fun()

In [ ]:
gen()

### Yield to count

In [ ]:
def counter():
    n = 0
    while True:
        n += 1`
        print('-> before yield')
        yield n
        print('-> after yield')

c = counter()

In [ ]:
print(next(c))

In [ ]:
print(c.send(None))

### Create coroutine with yield

You can **send** data to a coroutine.

In [ ]:
def simple_coro():
    print('-> coroutine started')
    x = yield
    print('-> coroutine recived', x)
    yield

**Prime** the coroutine

In [ ]:
simple = simple_coro()

In [ ]:
simple.send(None)

Send data

In [ ]:
simple.send(28)

A coroutine which **receives** data as well as **produces** data

In [ ]:
def averager():
    total, count, average = 0.0, 0, None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [ ]:
avg = averager()
next(avg)

In [ ]:
avg.send(10)

In [ ]:
avg.send(20)

In [ ]:
avg.send(30)

## WHY

### A tornado example

In [ ]:
import time
import tornado.ioloop
import tornado.web
import tornado.gen


class BadStupidHandler(tornado.web.RequestHandler):
    def get(self):
        for i in range(20):
            self.write('{}<br>'.format(i))
            self.flush()
            time.sleep(0.5)


class GoodStupidHandler(tornado.web.RequestHandler):
    @tornado.gen.coroutine
    def get(self):
        for i in range(20):
            self.write('{}<br>'.format(i))
            self.flush()
            yield tornado.gen.sleep(0.5)

app = tornado.web.Application([
     (r'/bad', BadStupidHandler),
     (r'/good', GoodStupidHandler)
])
app.listen(8001)
tornado.ioloop.IOLoop.current().start()


Refer to documentation [Frequently Asked Questions](http://www.tornadoweb.org/en/stable/faq.html).

## HOW

### Use coroutine to simplfy your context manager

A context manager is to change:

In [ ]:
try:
    f = open('coroutine.ipynb')
    f.readline() # do something with f
finally:
    f.close()

To:

In [ ]:
with open('coroutine.ipynb') as f:
    f.readline() # do something with f

You can define your own context manager with a **class** implements `__enter__` and `__exit__` method

In [ ]:
class Mirror:
    def __init__(self, num):
        self.num = num
    
    def __enter__(self):
        import sys
        
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'This is mirror {}'.format(self.num)
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
    
    def __exit__(self,exc_type, exc_value, traceback):
        import sys
        
        sys.stdout.write = self.original_write

In [ ]:
with Mirror(1000) as first_string:
    print(first_string)
    print('#',123456789,'#')
print('*',123456789,'*')

In [ ]:
with Mirror(1000) as first_string:
    print(first_string)
    with Mirror(2000) as second_string:
        print(second_string)

Use decorator `contextlib.contextmanager` and `generator` to simplfy your own context manager

In [ ]:
import contextlib

@contextlib.contextmanager
def Mirror_new(num):
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'This is mirror {}'.format(num)
    sys.stdout.write = original_write

In [ ]:
with Mirror_new(1000) as first_string:
    print(first_string)
    print(123456789)

How does it work?

We define a warpper class to proxy our generator:

In [ ]:
class  GeneratorCM:        
    def __init__(self, gen):
        self._gen = gen
        
    def __call__(self, *args, **kwargs):
        self._gen = self._gen(*args, **kwargs)
        return self
        
    def __enter__(self):
        return self._gen.send(None)
    
    def __exit__(self,exc_type, exc_value, traceback):
        try:
            self._gen.send(None)
        except StopIteration:
            return True

In [ ]:
@GeneratorCM
def Mirror_custom(num):
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'This is mirror {}'.format(num)
    sys.stdout.write = original_write

In [ ]:
with Mirror_custom(1000) as first_string:
    print(first_string)
    print(123456789)

GeneratorCM with full try catch is listed below:

In [ ]:
class  GeneratorCM:        
    def __init__(self, gen):
        self._gen = gen
        
    def __call__(self, *args, **kwargs):
        self._gen = self._gen(*args, **kwargs)
        return self
        
    def __enter__(self):
        return self._gen.send(None)
    
    def __exit__(self,exc_type, exc_value, traceback):
        try:
            if exc_type is None:
                next(self._gen)
            else:
                self._gen.throw(exc_type, exc_value, traceback)
                raise RuntimeError("Generator didn't stop")
        except StopIteration:
                return True
        except:
                if sys.exc_info()[1] is not val: raise

### Inlined yield

Following statement is very common in tornado framwork

In [ ]:
from tornado import gen

@gen.coroutine
def fetch_coroutine(url):
    http_client = AsyncHTTPClient()
    response = yield http_client.fetch(url)
    raise gen.Return(response.body)

How does it work?

First,define a slow function to mock `http.fetch` :

In [ ]:
def func(x, y):
    import time 
    time.sleep(3)
    return x + y

We need to our function work like this:

In [ ]:
from concurrent.futures import ThreadPoolExecutor
pool = ThreadPoolExecutor(max_workers=8)

## ADDITINAL